In [1]:
# DBA3701 Project
# By Joshua Emmanuel Teo Rui Zhong A0217683X
# This project aims to perform the optimisation of capacity planning in a last mile delivery context. 
# We want to deliver the most parcels with as little driver hours as possible. However, each of our drivers have different efficencies. 
# In addition, since they are more familiar with certain areas, their efficency varies based on postal district.

# Contents:
# Cell 1 (This cell) - Project Description
# Cell 2 - Imports
# Cell 3 - Miscellaneous Functions
# Cell 4 - Single Day Optimisation Model
# Cell 5 - Single Day Optimisation Analysis
# Cell 6 - Multi Day Optimisation Model
# Cell 7 - Multi Day Optimisation Analysis
# Cell 8 - Multi Day with Uncertainty Optimisation Model
# Cell 9 - Multi Day with Uncertainty Optimisation Analysis
# Cell 10 - Limitations

In [1]:
import numpy as np
import pandas as pd
import plotly.express as px
import random
from rsome import norm, ro
from rsome import grb_solver as grb

In [2]:
# Miscellaneous functions

# Randomizes the distribution percentage of parcels across all districts
def randomizeParcelDist(nDistricts):
    parcelDist = [random.randint(1, 10)  for i in range(nDistricts)]
    parcelDist = [i/sum(parcelDist) for i in parcelDist]
    return parcelDist

# Bonus efficiency of each driver is depedent on how familiar they are with that location. 
def bonusEfficiency(driver, district, base, nDistricts, nDrivers):
    # normalize driver and district
    normDriver, normDistrict = driver/nDrivers, district/nDistricts
    # add an element of randomness
    return round(base*(1 + abs(normDriver-normDistrict)*random.random()))


In [4]:
# Firstly, we optimise over a single day.

def optimizeSingleDay(nDistricts, nDrivers, totalParcels, costDelivered, costUndelivered, costDriver, maxHours, minDriverEfficiency, maxDriverEfficiency):
    """
    Optimizes capacity planning for a single day.

    Args:
        nDistricts (int): Number of districts to deliver to
        nDrivers (int): Number of drivers available
        totalParcels (int): Total number of parcels to deliver
        costDelivered (int): Amount earned per successful delivery
        costUndelivered (int): Cost per undelivered parcel
        costDriver (int): Hourly wage of each driver
        maxHours (int): Maximum number of hours each driver can work
        minDriverEfficiency (int): Lower bound of the efficiency of each driver (in terms of parcels per hour)
        maxDriverEfficiency (int): Upper bound of the efficiency of each driver (in terms of parcels per hour)

    Returns:
        model.get(): Maximum profit possible
    """
    
    # Static variables
    model = ro.Model()
    parcelDist = randomizeParcelDist(nDistricts)
    parcels = [round(i*totalParcels) for i in parcelDist]
    parcels[-1] = totalParcels - sum(parcels[:-1])
    
    baseEfficiency = [random.randint(minDriverEfficiency, maxDriverEfficiency) for i in range(nDrivers)]
    efficiencyMatrix = np.array([[bonusEfficiency(i, j, baseEfficiency[i], nDistricts, nDrivers) for i in range(nDrivers)] for j in range(nDistricts)])
    
    # Decision variables
    deliveringAmt = model.dvar((nDistricts, nDrivers), 'I') # 2D Matrix to keep track of the number of parcels each driver is delivering for each district
    deliveringHours = model.dvar((nDistricts, nDrivers), 'C') # 2D Matrix to keep track of the number of hours each driver is putting in for each district
    isDelivering = model.dvar((nDistricts, nDrivers), 'B') # 2D Matrix to keep track if a driver is delivering to a district
    
    # Objective
    model.max(deliveringAmt.sum()*costDelivered - (totalParcels - deliveringAmt.sum())*costUndelivered - deliveringHours.sum()*costDriver)

    # Constraints
    # Each driver cannot deliver more than his efficiency x hours
    model.st(0 <= deliveringAmt, deliveringAmt <= efficiencyMatrix*deliveringHours)
            
    # Each driver cannot drive more hours than the max allowed
    model.st(0 <= deliveringHours, deliveringHours <= isDelivering*maxHours)
        
    # Each driver can only deliver to 1 district
    model.st(isDelivering.sum(axis=0) <= 1)

    # Number of parcels delivered per region cannot exceed total number of parcels available to be delivered to that region
    model.st(deliveringAmt[i].sum() <= parcels[i] for i in range(nDistricts))

    model.solve(solver=grb, params={'LogToConsole': 0, 'OutputFlag': 0})
    return model.get()

In [5]:
# Now, we perform some analysis using our single day model.
# Some data as a reference point:
# Ninjavan has roughly 8710 drivers and delivers 260000 parcels daily in SG.
# Ref https://media.ninjavan.co/sg/wp-content/uploads/sites/9/2022/04/REPORT-Ninja-Van-Group-x-DPDGroup_-E-commerce-Barometer-Report-1.pdf
# For our single day model, we scale this down by 200x.
# (Some of the models may need to be ran multiple times as they may not be able to find an optimal solution every time.)
nIterations = 15 # Number of iterations we will be running the model

# First, we vary the number of drivers.
vars = [5,15,25,35,45,55]
profitsSingleDayVaryDrivers = {
    'name' : [],
    'iteration' : [],
    'profit' : []
}
for i in range(nIterations):
    for var in vars:
        res = optimizeSingleDay(
            nDistricts=28, 
            nDrivers=var, 
            totalParcels=1300, 
            costDelivered=10, 
            costUndelivered=8, 
            costDriver=20, 
            maxHours=8, 
            minDriverEfficiency=3, 
            maxDriverEfficiency=10
            )
        profitsSingleDayVaryDrivers['name'].append(str(var) + ' Drivers')
        profitsSingleDayVaryDrivers['iteration'].append(i)
        profitsSingleDayVaryDrivers['profit'].append(res)
        
df = pd.DataFrame(profitsSingleDayVaryDrivers)
fig = px.line(df, x='iteration', y='profit', color='name', labels={'iteration': "Iteration", 'profit': "Profit", 'name': "Category"}, title='Single Day Optimisation: Varying no. of Drivers')
fig.show()

# We can see that as the number of drivers increase, marginal profit gain per driver decreases. 
# This is because as the number of available parcels to deliver decreases, some drivers are used less optimally.
# We can also see that as the number of drivers increase, variance in profit decreases. 
# This is because parcels can be more reliably delivered optimally.

# Next, we vary the number of districts.
vars = [1,5,15,25,35,45]
profitsSingleDayVaryDistricts = {
    'name' : [],
    'iteration' : [],
    'profit' : []
}
for i in range(nIterations):
    for var in vars:
        res = optimizeSingleDay(
            nDistricts=var, 
            nDrivers=25, 
            totalParcels=1300, 
            costDelivered=10, 
            costUndelivered=8, 
            costDriver=20, 
            maxHours=8, 
            minDriverEfficiency=3, 
            maxDriverEfficiency=10
            )
        profitsSingleDayVaryDistricts['name'].append(str(var) + ' Districts')
        profitsSingleDayVaryDistricts['iteration'].append(i)
        profitsSingleDayVaryDistricts['profit'].append(res)
        
df = pd.DataFrame(profitsSingleDayVaryDistricts)
fig = px.line(df, x='iteration', y='profit', color='name', labels={'iteration': "Iteration", 'profit': "Profit", 'name': "Category"}, title='Single Day Optimisation: Varying no. of Districts')
fig.show()

# We can see that as the number of districts increase (keeping total drivers and parcels constant), profit decreases.
# This is because past a certain threshold, parcels are spread too thin and drivers cannot be assigned optimally.

# Lastly, we vary the upper limit of our driver's efficiency.
vars = [5,10,15,20,25,30]
profitsSingleDayVaryDEUL = {
    'name' : [],
    'iteration' : [],
    'profit' : []
}
for i in range(nIterations):
    for var in vars:
        res = optimizeSingleDay(
            nDistricts=28, 
            nDrivers=25, 
            totalParcels=1300, 
            costDelivered=10, 
            costUndelivered=8, 
            costDriver=20, 
            maxHours=8, 
            minDriverEfficiency=3, 
            maxDriverEfficiency=var
            )
        profitsSingleDayVaryDEUL['name'].append('Upper Limit of Driver Efficiency: ' + str(var))
        profitsSingleDayVaryDEUL['iteration'].append(i)
        profitsSingleDayVaryDEUL['profit'].append(res)
        
df = pd.DataFrame(profitsSingleDayVaryDEUL)
fig = px.line(df, x='iteration', y='profit', color='name', labels={'iteration': "Iteration", 'profit': "Profit", 'name': "Category"}, title='Single Day Optimisation: Varying Driver Efficiency')
fig.show()

# We can see that as driver efficiency increases, profit increases, and variance decreases.
# This is because drivers can more reliably deliver all parcels optimally.

Set parameter Username
Academic license - for non-commercial use only - expires 2023-10-26
Being solved by Gurobi...
Solution status: 2
Running time: 0.0472s
Being solved by Gurobi...
Solution status: 2
Running time: 0.4248s
Being solved by Gurobi...
Solution status: 2
Running time: 0.2953s
Being solved by Gurobi...
Solution status: 2
Running time: 0.1631s
Being solved by Gurobi...
Solution status: 2
Running time: 0.0890s
Being solved by Gurobi...
Solution status: 2
Running time: 0.1980s
Being solved by Gurobi...
Solution status: 2
Running time: 0.0147s
Being solved by Gurobi...
Solution status: 2
Running time: 0.5876s
Being solved by Gurobi...
Solution status: 2
Running time: 0.2621s
Being solved by Gurobi...
Solution status: 2
Running time: 0.1579s
Being solved by Gurobi...
Solution status: 2
Running time: 0.0928s
Being solved by Gurobi...
Solution status: 2
Running time: 0.1293s
Being solved by Gurobi...
Solution status: 2
Running time: 0.0771s
Being solved by Gurobi...
Solution sta

Being solved by Gurobi...
Solution status: 2
Running time: 0.0955s
Being solved by Gurobi...
Solution status: 2
Running time: 0.0977s
Being solved by Gurobi...
Solution status: 2
Running time: 2.7397s
Being solved by Gurobi...
Solution status: 2
Running time: 0.0992s
Being solved by Gurobi...
Solution status: 2
Running time: 0.1703s
Being solved by Gurobi...
Solution status: 2
Running time: 0.1013s
Being solved by Gurobi...
Solution status: 2
Running time: 0.0063s
Being solved by Gurobi...
Solution status: 2
Running time: 0.3757s
Being solved by Gurobi...
Solution status: 2
Running time: 2.0109s
Being solved by Gurobi...
Solution status: 2
Running time: 18.4996s
Being solved by Gurobi...
Solution status: 2
Running time: 0.2280s
Being solved by Gurobi...
Solution status: 2
Running time: 0.4334s
Being solved by Gurobi...
Solution status: 2
Running time: 0.1334s
Being solved by Gurobi...
Solution status: 2
Running time: 0.4795s
Being solved by Gurobi...
Solution status: 2
Running time: 7.

Being solved by Gurobi...
Solution status: 2
Running time: 6.8264s
Being solved by Gurobi...
Solution status: 2
Running time: 0.1041s
Being solved by Gurobi...
Solution status: 2
Running time: 0.1165s
Being solved by Gurobi...
Solution status: 2
Running time: 0.0698s
Being solved by Gurobi...
Solution status: 2
Running time: 0.0508s
Being solved by Gurobi...
Solution status: 2
Running time: 0.1142s
Being solved by Gurobi...
Solution status: 2
Running time: 12.1787s
Being solved by Gurobi...
Solution status: 2
Running time: 0.1247s
Being solved by Gurobi...
Solution status: 2
Running time: 0.1079s
Being solved by Gurobi...
Solution status: 2
Running time: 0.0720s
Being solved by Gurobi...
Solution status: 2
Running time: 0.1476s
Being solved by Gurobi...
Solution status: 2
Running time: 0.1351s
Being solved by Gurobi...
Solution status: 2
Running time: 2.4096s
Being solved by Gurobi...
Solution status: 2
Running time: 0.2321s
Being solved by Gurobi...
Solution status: 2
Running time: 0.

In [6]:
# In reality, parcels not delivered at the end of the day are left to the next day.
# Now, we introduce the dimension of time.

def optimizeOverPeriod(nDays, nDistricts, nDrivers, totalParcelsAvg, totalParcelsSD, costDelivered, costUndelivered, costDriver, maxHours, minDriverEfficiency, maxDriverEfficiency):
    """
    Optimizes capacity planning over a period of time.

    Args:
        nDays (int): Number of days to run optimizer over
        nDistricts (int): Number of districts to deliver to
        nDrivers (int): Number of drivers available
        totalParcelsAvg (int): Average number of parcels to deliver each day
        totalParcelsSD (int): Standard deviation of the number of parcels to deliver each day
        costDelivered (int): Amount earned per successful delivery
        costUndelivered (int): Cost per undelivered parcel at the end of every day
        costDriver (int): Hourly wage of each driver
        maxHours (int): Maximum number of hours each driver can work each day
        minDriverEfficiency (int): Lower bound of the efficiency of each driver (in terms of parcels per hour)
        maxDriverEfficiency (int): Upper bound of the efficiency of each driver (in terms of parcels per hour)

    Returns:
        model.get(): Maximum profit possible
        profits.get(): Profits for each day
    """
    
    # Static Variables
    baseParcels = [round(np.random.normal(totalParcelsAvg,totalParcelsSD)) for i in range(nDays)]
    parcelDist = randomizeParcelDist(nDistricts)
    
    baseEfficiency = [random.randint(minDriverEfficiency, maxDriverEfficiency) for i in range(nDrivers)]
    efficiencyMatrix = np.array([[bonusEfficiency(i, j, baseEfficiency[i], nDistricts, nDrivers) for i in range(nDrivers)] for j in range(nDistricts)])
    efficiencyMatrix = np.tile(efficiencyMatrix, [nDays,1,1])
    
    model = ro.Model()

    # Decision Variables
    deliveringAmt = model.dvar((nDays, nDistricts, nDrivers), 'I') # 3D Matrix to keep track of the number of parcels each driver is delivering to each district each day
    deliveringHours = model.dvar((nDays, nDistricts, nDrivers), 'C') # 3D Matrix to keep track of the number of hours each driver is putting in for each district each day
    isDelivering = model.dvar((nDays, nDistricts, nDrivers), 'B') # 3D Matrix to keep track if a driver is delivering to a district each day
    undeliveredParcels = model.dvar(nDays, 'I') # List to track the number of undelivered parcels each day
    profits = model.dvar(nDays, 'C') # Keeps track of the profits of each day
    
    # Objective
    model.max(profits.sum())

    # Constraints
    # Daily profit depends on number of parcels succesfully delivered and number of hours worked across all drivers
    model.st(profits[i] <= deliveringAmt[i].sum()*costDelivered - undeliveredParcels[i]*costUndelivered - deliveringHours[i].sum()*costDriver for i in range(nDays))
    
    # Parcels to be delivered each day is calculated with that day's base + ytd leftover
    model.st(undeliveredParcels[0] >= baseParcels[0]-deliveringAmt[0].sum())
    model.st(undeliveredParcels[i] >= baseParcels[i]+undeliveredParcels[i-1]-deliveringAmt[i].sum() for i in range(1, nDays))

    # Each driver cannot deliver more than his efficiency x hours
    model.st(0 <= deliveringAmt, deliveringAmt <= efficiencyMatrix*deliveringHours)
            
    # Each driver cannot drive more hours than the max allowed
    model.st(0 <= deliveringHours, deliveringHours <= isDelivering*maxHours)
        
    # Each driver can only deliver to 1 district
    model.st(isDelivering[i].sum(axis=0) <= 1 for i in range(nDays))

    # Number of parcels delivered per region cannot exceed total number of parcels available to be delivered to that region
    for i in range(nDays):
        for j in range(nDistricts):
            if (i == 0):
                model.st(deliveringAmt[i][j].sum() <= baseParcels[i]*parcelDist[j])
            else:
                model.st(deliveringAmt[i][j].sum() <= (baseParcels[i]+undeliveredParcels[i-1])*parcelDist[j])

    model.solve(solver=grb, params={'LogToConsole': 0, 'OutputFlag': 0})
    return model.get(), profits.get()

In [7]:
# Now, we perform some analysis using our multi day model.
# (Some of the models may need to be ran multiple times as they may not be able to find an optimal solution every time.)

# First, we vary the number of drivers.
vars = [1,5,10,15,25,30]
profitsMultiDayVaryDrivers = {
    'name' : [],
    'day' : [],
    'profit' : []
}

for var in vars:
    res, resProfits = optimizeOverPeriod(
        nDays=5,
        nDistricts=5, 
        nDrivers=var, 
        totalParcelsAvg=1300,
        totalParcelsSD=200, 
        costDelivered=10, 
        costUndelivered=8, 
        costDriver=20, 
        maxHours=8, 
        minDriverEfficiency=3, 
        maxDriverEfficiency=10
        )
    for i in range(len(resProfits)):
        profitsMultiDayVaryDrivers['name'].append(str(var) + ' Drivers')
        profitsMultiDayVaryDrivers['day'].append(i+1)
        profitsMultiDayVaryDrivers['profit'].append(resProfits[i])
        
df = pd.DataFrame(profitsMultiDayVaryDrivers)
fig = px.line(df, x='day', y='profit', color='name', labels={'day': "Day", 'profit': "Profit", 'name': "Category"}, title='Multi Day Optimisation: Varying no. of Drivers')
fig.show()

# We can see that as time progresses, the undelivered parcels stockpile and this results in decreasing profit.
# We can also see that as the number of drivers increase, marginal profit gain per driver and variance remains relatively constant, contrasting to our single day model.
# When parcels are carried over to the next day, there are more parcels to be delivered, hence drivers can be more consistently utilised more optimally.

# Next, we vary the number of districts.
vars = [2,4,6,9,12,15]
profitsMultiDayVaryDistricts = {
    'name' : [],
    'day' : [],
    'profit' : []
}

for var in vars:
    res, resProfits = optimizeOverPeriod(
        nDays=5,
        nDistricts=var, 
        nDrivers=10, 
        totalParcelsAvg=850,
        totalParcelsSD=85, 
        costDelivered=10, 
        costUndelivered=8, 
        costDriver=20, 
        maxHours=8, 
        minDriverEfficiency=3, 
        maxDriverEfficiency=10
        )
    for i in range(len(resProfits)):
        profitsMultiDayVaryDistricts['name'].append(str(var) + ' Districts')
        profitsMultiDayVaryDistricts['day'].append(i+1)
        profitsMultiDayVaryDistricts['profit'].append(resProfits[i])
        
df = pd.DataFrame(profitsMultiDayVaryDistricts)
fig = px.line(df, x='day', y='profit', color='name', labels={'day': "Day", 'profit': "Profit", 'name': "Category"}, title='Multi Day Optimisation: Varying no. of Districts')
fig.show()

# Interestingly enough, there are no meaningful trends here, as compared to our single day model.

Being solved by Gurobi...
Solution status: 2
Running time: 0.0010s
Being solved by Gurobi...
Solution status: 2
Running time: 0.0480s
Being solved by Gurobi...
Solution status: 2
Running time: 0.0521s
Being solved by Gurobi...
Solution status: 2
Running time: 3.1841s
Being solved by Gurobi...
Solution status: 2
Running time: 67.5830s
Being solved by Gurobi...
Solution status: 2
Running time: 44.1103s


Being solved by Gurobi...
Solution status: 2
Running time: 0.0472s
Being solved by Gurobi...
Solution status: 2
Running time: 0.6570s
Being solved by Gurobi...
Solution status: 2
Running time: 0.3445s
Being solved by Gurobi...
Solution status: 2
Running time: 3.4048s
Being solved by Gurobi...
Solution status: 2
Running time: 3.5251s
Being solved by Gurobi...
Solution status: 2
Running time: 108.0103s


In [3]:
# Finally, we want to add an element of uncertainty, as in reality, some parcels are returned even though they have been successfully delivered.

def optimizeOverPeriodWithReturn(returnRate, nDays, nDistricts, nDrivers, totalParcelsAvg, totalParcelsSD, costDelivered, costUndelivered, costDriver, maxHours, minDriverEfficiency, maxDriverEfficiency):
    """
    Optimizes capacity planning over a period of time, factoring in possible returning of delivered packages.

    Args:
        returnRate (float): Average return rate of a delivered parcel. In decimal form
        nDays (int): Number of days to run optimizer over
        nDistricts (int): Number of districts to deliver to
        nDrivers (int): Number of drivers available
        totalParcelsAvg (int): Average number of parcels to deliver each day
        totalParcelsSD (int): Standard deviation of the number of parcels to deliver each day
        costDelivered (int): Amount earned per successful delivery
        costUndelivered (int): Cost per undelivered parcel at the end of every day
        costDriver (int): Hourly wage of each driver
        maxHours (int): Maximum number of hours each driver can work each day
        minDriverEfficiency (int): Lower bound of the efficiency of each driver (in terms of parcels per hour)
        maxDriverEfficiency (int): Upper bound of the efficiency of each driver (in terms of parcels per hour)

    Returns:
        model.get(): Maximum profit possible
        profits.get(): Profits for each day
    """
    # Static Variables
    baseParcels = [round(np.random.normal(totalParcelsAvg,totalParcelsSD)) for i in range(nDays)]
    parcelDist = randomizeParcelDist(nDistricts)
    
    baseEfficiency = [random.randint(minDriverEfficiency, maxDriverEfficiency) for i in range(nDrivers)]
    efficiencyMatrix = np.array([[bonusEfficiency(i, j, baseEfficiency[i], nDistricts, nDrivers) for i in range(nDrivers)] for j in range(nDistricts)])
    efficiencyMatrix = np.tile(efficiencyMatrix, [nDays,1,1])
    
    model = ro.Model()

    # Decision Variables
    deliveringAmt = model.dvar((nDays, nDistricts, nDrivers), 'I') # 3D Matrix to keep track of the number of parcels each driver is delivering to each district each day
    deliveringHours = model.dvar((nDays, nDistricts, nDrivers), 'C') # 3D Matrix to keep track of the number of hours each driver is putting in for each district each day
    isDelivering = model.dvar((nDays, nDistricts, nDrivers), 'B') # 3D Matrix to keep track if a driver is delivering to a district each day
    undeliveredParcels = model.dvar(nDays, 'I') # List to track the number of undelivered parcels each day
    profits = model.dvar(nDays, 'C') # Keeps track of the profits of each day
    returnChance = model.rvar(nDays, 'C') # Tracks the return rate of a parcel being returned each day.
    uset = (0 <= returnChance, norm(returnChance, np.infty) <= 1, norm(returnChance, 1) <= returnRate*nDays) # 0 <= returnChance <= 1, E(returnChance) == returnRate
    
    # Objective
    model.maxmin(profits.sum(), uset)

    # Constraints
    # Daily profit depends on number of parcels succesfully delivered and number of hours worked across all drivers
    model.st(profits[i] <= deliveringAmt[i].sum()*costDelivered - undeliveredParcels[i]*costUndelivered - deliveringHours[i].sum()*costDriver for i in range(nDays))
    
    # Parcels to be delivered each day is calculated with that day's base + ytd leftover
    model.st(undeliveredParcels[0] >= baseParcels[0]-deliveringAmt[0].sum())
    model.st(undeliveredParcels[i] >= baseParcels[i]+undeliveredParcels[i-1]-deliveringAmt[i].sum() for i in range(1, nDays))

    # Each driver cannot deliver more than his efficiency x hours, factoring in return chance
    for i in range(nDays):
        for j in range(nDistricts):
            for k in range(nDrivers):
                model.st(0 <= deliveringAmt[i][j][k], (deliveringAmt[i][j][k] <= efficiencyMatrix[i][j][k]*deliveringHours[i][j][k]*(1-returnChance[i])))
            
    # Each driver cannot drive more hours than the max allowed
    model.st(0 <= deliveringHours, deliveringHours <= isDelivering*maxHours)
        
    # Each driver can only deliver to 1 district
    model.st(isDelivering[i].sum(axis=0) <= 1 for i in range(nDays))

    # Number of parcels delivered per region cannot exceed total number of parcels available to be delivered to that region
    for i in range(nDays):
        for j in range(nDistricts):
            if (i == 0):
                model.st(deliveringAmt[i][j].sum() <= baseParcels[i]*parcelDist[j])
            else:
                model.st(deliveringAmt[i][j].sum() <= (baseParcels[i]+undeliveredParcels[i-1])*parcelDist[j])

    model.solve(solver=grb, params={'LogToConsole': 0, 'OutputFlag': 0})
    return model.get(), profits.get()

In [4]:
# Now, we perform some analysis using our multi day with return chance model.
# Some data as a reference point:
# Ninjavan has an approximate return rate of 24% in SG.
# Ref: https://media.ninjavan.co/sg/wp-content/uploads/sites/9/2022/04/REPORT-Ninja-Van-Group-x-DPDGroup_-E-commerce-Barometer-Report-1.pdf
# (Some of the models may need to be ran multiple times as they may not be able to find an optimal solution every time.)

# Varying the return rate.
vars = [0.05, 0.1, 0.15, 0.2, 0.3, 0.5]
profitsUncertaintyVaryReturnRate = {
    'name' : [],
    'day' : [],
    'profit' : []
}

for var in vars:
    res, resProfits = optimizeOverPeriodWithReturn(
        returnRate=var,
        nDays=7,
        nDistricts=5, 
        nDrivers=20, 
        totalParcelsAvg=650,
        totalParcelsSD=65, 
        costDelivered=10, 
        costUndelivered=8, 
        costDriver=20, 
        maxHours=8, 
        minDriverEfficiency=3, 
        maxDriverEfficiency=10
        )
    for i in range(len(resProfits)):        
        profitsUncertaintyVaryReturnRate['name'].append(str(var) + ' Return Rate')
        profitsUncertaintyVaryReturnRate['day'].append(i+1)
        profitsUncertaintyVaryReturnRate['profit'].append(resProfits[i])
        
df = pd.DataFrame(profitsUncertaintyVaryReturnRate)
fig = px.line(df, x='day', y='profit', color='name', labels={'day': "Day", 'profit': "Profit", 'name': "Category"}, title='Multi Day Optimisation with Return: Varying Return Rate')
fig.show()

# As expected, a drop in delivery success rate directly translates to a drop in profit.
# However, we can see that depending on the number of available drivers, we can prepare for and withstand a given level of uncertainty.
# We also note that uncertainty rate does not affect variance.

Set parameter Username
Academic license - for non-commercial use only - expires 2023-10-26
Being solved by Gurobi...
Solution status: 2
Running time: 74.6252s
Being solved by Gurobi...
Solution status: 2
Running time: 1.9939s
Being solved by Gurobi...
Solution status: 2
Running time: 0.0908s
Being solved by Gurobi...
Solution status: 2
Running time: 0.0951s
Being solved by Gurobi...
Solution status: 2
Running time: 0.1169s
Being solved by Gurobi...
Solution status: 2
Running time: 0.0804s


In [ ]:
# Limitations

# I was unable to run this on a larger scale. 
# Interestingly, it seems to spike in solving time when a 'stable' profit stream is achieved, i.e. when the number of drivers is enough to meet the number of parcels. 
# For example, 
# nDrivers = 5, nDrivers not sufficient to deliver all parcels. Parcels keep piling up, and profits keep decreasing. Solving time < 1s.
# nDrivers = 7, nDrivers can now consistently deliver all parcels. Daily profit is relatively stable. Solving time > 60s / cannot solve.
# One possible reason I could think of is that when a 'stable' profit stream is possible, there are less parcels to deliver each day.
# Thus it is harder to assign drivers optimally, which causes a significant spike when compounded across many days.